## Open notebook in:
| Colab                                 |  Gradient    [link text](https://)                                                                                                                                     |
|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Nicolepcx/transformers-the-definitive-guide/blob/master/CH04/CH04_patch_tst_hyperparameter_IBM_10_days_ahead_32_context_window.ipynb)                                             | [![Gradient](https://assets.paperspace.io/img/gradient-badge.svg)](https://console.paperspace.com//github.com/Nicolepcx/transformers-the-definitive-guide/blob/main/CH04/CH04_patch_tst_hyperparameter_IBM_10_days_ahead_32_context_window.ipynb)|             

# About this Notebook

This notebook provides an overview on how to use the `PatchTST` model from the Hugging Face `transformers` library. The workflow involves loading and preprocessing time series data, configuring the model, performing hyperparameter tuning with Optuna, and finally training and evaluating the model.

### Steps Included:

1. **Setting Up the Environment**:  
   Import necessary libraries, suppress warnings, and set a random seed to ensure reproducibility.

2. **Loading and Preparing the Dataset**:  
   Load time series data (e.g., stock prices) into a Pandas DataFrame. The data is then split into training, validation, and test sets based on specified indices. The `TimeSeriesPreprocessor` is used to normalize and prepare the data for model training.

3. **Configuring the PatchTST Model**:  
   Define the `PatchTSTConfig` configuration for the model, specifying parameters such as the number of input channels, context length, patch length, and the model's architecture. The model is initialized using this configuration.

4. **Hyperparameter Tuning with Optuna**:  
   Use Optuna to perform a hyperparameter search, exploring different configurations of learning rate, batch size, number of epochs, and other parameters. The goal is to minimize the evaluation loss, and early stopping is implemented to prevent overfitting.

5. **Training the Model**:  
   The `Trainer` class from `transformers` is used to handle the training loop. The best hyperparameters found during the Optuna search are applied to the training arguments, and the model is trained on the time series data.

6. **Model Evaluation**:  
   After training, the model is evaluated on the validation and test datasets. The results, including evaluation metrics such as loss, are printed to assess the model's performance.

7. **Saving the Model**:  
   Finally, the trained model and its configurations are saved for future use, allowing for easy deployment and inference on new time series data.

The notebook is partially based on the example code from the original [paper's repo](https://github.com/yuqinie98/PatchTST).


In [1]:
!pip install git+https://github.com/IBM/tsfm.git -qqq

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 116.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.9/143.9 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 81.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.

In [2]:
# Standard
import os
import numpy as np
import pandas as pd

# Third Party
from transformers import (
    EarlyStoppingCallback,
    PatchTSTConfig,
    PatchTSTForPrediction,
    set_seed,
    Trainer,
    TrainingArguments,
)
import optuna
import yfinance as yf

# First Party
from tsfm_public.toolkit.dataset import ForecastDFDataset
from tsfm_public.toolkit.time_series_preprocessor import TimeSeriesPreprocessor
from tsfm_public.toolkit.util import select_by_index

# Supress some warnings
import warnings

warnings.filterwarnings("ignore", module="torch")

### Set seed

In [3]:
set_seed(42)

# Load and prepare datasets

In the next cell, please adjust the following parameters to suit your application:
- `PRETRAIN_AGAIN`: Set this to `True` if you want to perform pretraining again. Note that this might take some time depending on the GPU availability. Otherwise, the already pretrained model will be used.
- `dataset_path`: path to local .csv file, or web address to a csv file for the data of interest. Data is loaded with pandas, so anything supported by
`pd.read_csv` is supported: (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html).
- `timestamp_column`: column name containing timestamp information, use None if there is no such column
- `id_columns`: List of column names specifying the IDs of different time series. If no ID column exists, use []
- `forecast_columns`: List of columns to be modeled
- `context_length`: The amount of historical data used as input to the model. Windows of the input time series data with length equal to
`context_length` will be extracted from the input dataframe. In the case of a multi-time series dataset, the context windows will be created
so that they are contained within a single time series (i.e., a single ID).
- `forecast_horizon`: Number of timestamps to forecast in future.
- `train_start_index`, `train_end_index`: the start and end indices in the loaded data which delineate the training data.
- `valid_start_index`, `valid_end_index`: the start and end indices in the loaded data which delineate the validation data.
- `test_start_index`, `test_end_index`: the start and end indices in the loaded data which delineate the test data.
- `patch_length`: The patch length for the `PatchTSMixer` model. It is recommended to choose a value that evenly divides `context_length`.
- `num_workers`: Number of dataloder workers in pytorch dataloader.
- `batch_size`: Batch size.
The data is first loaded into a Pandas dataframe and split into training, validation, and test parts. Then the pandas dataframes are converted
to the appropriate torch dataset needed for training.

In [4]:
data = yf.download("IBM", start=None, end="2024-07-01")

[*********************100%%**********************]  1 of 1 completed


In [5]:
data = data[['Adj Close']]

In [6]:
data = data.reset_index()

In [7]:
data = data.rename(columns={'Adj Close': 'close', 'Date': 'date'})
data

,date,close
0,1962-01-02,1.513321
1,1962-01-03,1.526550
2,1962-01-04,1.511336
3,1962-01-05,1.481573
4,1962-01-08,1.453794
...,...,...
15725,2024-06-24,173.492584
15726,2024-06-25,171.103500
15727,2024-06-26,170.379822
15728,2024-06-27,169.368668


In [8]:
timestamp_column = "date"
id_columns = []

context_length = 32
forecast_horizon = 10
patch_length = 8
num_workers = 16
batch_size = 128

In [9]:

forecast_columns = list(data.columns[1:])

# get split
num_train = int(len(data) * 0.7)
num_test = int(len(data) * 0.2)
num_valid = len(data) - num_train - num_test
border1s = [
    0,
    num_train - context_length,
    len(data) - num_test - context_length,
]
border2s = [num_train, num_train + num_valid, len(data)]

train_start_index = border1s[0]  # None indicates beginning of dataset
train_end_index = border2s[0]

valid_start_index = train_end_index + context_length
valid_end_index = border2s[1]

test_start_index = valid_end_index + context_length
test_end_index = border2s[2]

train_data = select_by_index(
    data,
    id_columns=id_columns,
    start_index=train_start_index,
    end_index=train_end_index,
)

valid_data = select_by_index(
    data,
    id_columns=id_columns,
    start_index=valid_start_index,
    end_index=valid_end_index,
)
test_data = select_by_index(
    data,
    id_columns=id_columns,
    start_index=test_start_index,
    end_index=test_end_index,
)

tsp = TimeSeriesPreprocessor(
    timestamp_column=timestamp_column,
    id_columns=id_columns,
    target_columns=forecast_columns,
    scaling=True,
)
tsp = tsp.train(train_data)

In [10]:
print("Training Data Range: {} to {}".format(train_start_index, train_end_index))
print("Validation Data Range: {} to {}".format(valid_start_index, valid_end_index))
print("Testing Data Range: {} to {}".format(test_start_index, test_end_index))


Training Data Range: 0 to 11011
Validation Data Range: 11043 to 12584
Testing Data Range: 12616 to 15730


In [11]:
train_dataset = ForecastDFDataset(
    tsp.preprocess(train_data),
    id_columns=id_columns,
    timestamp_column="date",
    target_columns=forecast_columns,
    context_length=context_length,
    prediction_length=forecast_horizon,
)
valid_dataset = ForecastDFDataset(
    tsp.preprocess(valid_data),
    id_columns=id_columns,
    timestamp_column="date",
    target_columns=forecast_columns,
    context_length=context_length,
    prediction_length=forecast_horizon,
)
test_dataset = ForecastDFDataset(
    tsp.preprocess(test_data),
    id_columns=id_columns,
    timestamp_column="date",
    target_columns=forecast_columns,
    context_length=context_length,
    prediction_length=forecast_horizon,
)

# Configure the Model

In [12]:
config = PatchTSTConfig(
    num_input_channels=len(forecast_columns),
    context_length=context_length,
    patch_length=patch_length,
    patch_stride=patch_length,
    prediction_length=forecast_horizon,
    random_mask_ratio=0.4,
    d_model=16,
    num_attention_heads=8,
    num_hidden_layers=3,
    ffn_dim=256,
    dropout=0.2,
    head_dropout=0.2,
    pooling_type=None,
    channel_attention=False,
    scaling="std",
    loss="mse",
    pre_norm=True,
    norm_type="batchnorm",
)
model = PatchTSTForPrediction(config)

# Configure Hyperparameter Tuning

In [14]:
def optuna_hp_space(trial: optuna.Trial):
    return {
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-8, 1e-2),  # Granular range
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [16, 32, 64, 128]),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 50, 300, step=20),
        "dataloader_num_workers": trial.suggest_int("dataloader_num_workers", 0, 16, step=4),
        "weight_decay": trial.suggest_float("weight_decay", 0.0, 0.3, step=0.05),  # Granular steps for weight decay
        "per_device_eval_batch_size": trial.suggest_categorical("per_device_eval_batch_size", [16, 32, 64, 128]),
    }


In [15]:
def model_init(trial):
    return PatchTSTForPrediction(config)

In [16]:
training_args = TrainingArguments(
    output_dir="./checkpoint/output_dir",
    overwrite_output_dir=True,
    do_eval=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    save_total_limit=3,
    logging_dir="./checkpoint/logging_dir",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    num_train_epochs=200,  # Note: The actual number of epochs might be lower due to early stopping
    label_names=["future_values"],
)

trainer = Trainer(
    model=None,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    model_init=model_init,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=30, early_stopping_threshold=0.00001)]
)

# Start the hyperparameter search
best_run = trainer.hyperparameter_search(
    backend="optuna",
    n_trials=30,
    direction="minimize",
)

print("Best run:", best_run)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
[I 2024-08-22 15:24:02,407] A new study created in memory with name: no-name-b4dc2eb4-3cbb-4398-bd26-300143258197


Epoch,Training Loss,Validation Loss
1,0.011400,0.043518
2,0.011400,0.043366


[I 2024-08-22 15:25:19,595] Trial 0 finished with value: 0.04336557909846306 and parameters: {'learning_rate': 2.0841419105990525e-06, 'num_train_epochs': 2, 'seed': 28, 'per_device_train_batch_size': 4}. Best is trial 0 with value: 0.04336557909846306.


Epoch,Training Loss,Validation Loss
1,0.011200,0.040872
2,0.010200,0.038104


[I 2024-08-22 15:26:34,883] Trial 1 finished with value: 0.03810364007949829 and parameters: {'learning_rate': 7.5795804826512866e-06, 'num_train_epochs': 2, 'seed': 28, 'per_device_train_batch_size': 4}. Best is trial 1 with value: 0.03810364007949829.


...

Epoch,Training Loss,Validation Loss
1,0.011400,0.043006


[I 2024-08-22 15:36:36,345] Trial 28 pruned. 


Epoch,Training Loss,Validation Loss
1,0.009500,0.023589
2,0.004800,0.018841
3,0.004400,0.018759
4,0.004300,0.018556


[I 2024-08-22 15:39:09,388] Trial 29 finished with value: 0.018555982038378716 and parameters: {'learning_rate': 1.632402134814743e-05, 'num_train_epochs': 4, 'seed': 27, 'per_device_train_batch_size': 4}. Best is trial 11 with value: 0.018153363838791847.


Best run: BestRun(run_id='11', objective=0.018153363838791847, hyperparameters={'learning_rate': 9.171180859945352e-05, 'num_train_epochs': 5, 'seed': 40, 'per_device_train_batch_size': 32}, run_summary=None)


# Train Model on Best Hyperparameters

In [17]:

best_hyperparameters = best_run.hyperparameters

# Update training arguments with the best hyperparameters
training_args = TrainingArguments(
    output_dir="./checkpoint/output_dir",
    overwrite_output_dir=True,
    learning_rate=best_hyperparameters['learning_rate'],
    per_device_train_batch_size=int(best_hyperparameters['per_device_train_batch_size']),  # Make sure to cast to int if needed
    do_eval=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    save_total_limit=3,
    logging_dir="./checkpoint/logging_dir",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    num_train_epochs=200,  # This can be adjusted based on your previous experience
    label_names=["future_values"],
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [18]:
# Reinitialize the Trainer with the updated arguments
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=30, early_stopping_threshold=0.00001)]
)

# Train the model with the best hyperparameters
trainer.train()


Epoch,Training Loss,Validation Loss
1,0.007100,0.020322
2,0.004300,0.018434
3,0.004100,0.018876
4,0.004100,0.018106
5,0.003900,0.018269
6,0.004000,0.018509
7,0.004000,0.018040
8,0.004000,0.018311
9,0.003900,0.018649
10,0.004000,0.018332


TrainOutput(global_step=15092, training_loss=0.003991606852877466, metrics={'train_runtime': 304.8006, 'train_samples_per_second': 7198.148, 'train_steps_per_second': 225.065, 'total_flos': 2752990479360.0, 'train_loss': 0.003991606852877466, 'epoch': 44.0})

# Display results

In [19]:
results_valid_dataset = trainer.evaluate(valid_dataset)
print("Valid Results:", results_valid_dataset)
results_test_dataset = trainer.evaluate(test_dataset)
print("Test Results:", results_test_dataset)


Valid Results: {'eval_loss': 0.017921317368745804, 'eval_runtime': 0.9947, 'eval_samples_per_second': 1507.974, 'eval_steps_per_second': 188.999, 'epoch': 44.0}
Test Results: {'eval_loss': 0.04856698215007782, 'eval_runtime': 2.5625, 'eval_samples_per_second': 1199.2, 'eval_steps_per_second': 150.241, 'epoch': 44.0}


In [20]:
results_valid_dataset

{'eval_loss': 0.017921317368745804,
 'eval_runtime': 0.9947,
 'eval_samples_per_second': 1507.974,
 'eval_steps_per_second': 188.999,
 'epoch': 44.0}